In [1]:
path = "MDA/Termin 7/Data/df_all.csv" #path 是文件路径  

In [2]:
from uutils import data_importer

In [7]:
import geopandas as gpd
import pandas as pd
from datetime import datetime #python 自带的包，跟geopandas ,pandas 一样
import glob

In [22]:
gdf = data_importer(path)

Importing data...


C:\Users\moxi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Import complete! Wait a little moment for some other settings...
All complete!


In [24]:
gdf

,datetime,altitude,days_elapsed,latitude,longitude,mode,user_id,distance,speed,acceleration,geometry
0,2008-03-28 14:54:40,-777.0,39535.621296,39.894178,116.318200,train,010,0.000000,0.000000,0.000000e+00,POINT (116.31820 39.89418)
1,2008-03-28 14:55:14,-777.0,39535.621690,39.894505,116.321132,train,010,253.375124,26.827954,2.191826e-01,POINT (116.32113 39.89451)
2,2008-03-28 14:56:13,-777.0,39535.622373,39.894953,116.326452,train,010,457.703187,27.927652,5.177485e-03,POINT (116.32645 39.89495)
3,2008-03-28 14:57:12,-777.0,39535.623056,39.894600,116.332542,train,010,522.318591,31.870287,1.856231e-02,POINT (116.33254 39.89460)
4,2008-03-28 14:58:11,-777.0,39535.623738,39.889622,116.337040,train,010,673.422399,41.090180,4.340816e-02,POINT (116.33704 39.88962)
...,...,...,...,...,...,...,...,...,...,...,...
5351928,2008-11-29 02:29:29,289.0,39781.103808,40.029320,116.411975,subway,179,23.206984,41.772572,-2.795354e-02,POINT (116.41197 40.02932)
5351929,2008-11-29 02:29:31,275.0,39781.103831,40.029111,116.411963,subway,179,23.228950,41.812110,5.491364e-03,POINT (116.41196 40.02911)
5351930,2008-11-29 02:29:33,274.0,39781.103854,40.028904,116.411962,subway,179,22.984443,41.371997,-6.112673e-02,POINT (116.41196 40.02890)
5351931,2008-11-29 02:29:35,274.0,39781.103877,40.028697,116.411961,subway,179,22.984442,41.371996,-2.061781e-07,POINT (116.41196 40.02870)


In [25]:
print(gdf['mode'].unique())

['train' 'taxi' 'walk' 'bus' 'subway' 'airplane' 'car' 'bike' 'boat' 'run'
 'motorcycle']


In [26]:
# Separat independent features and dependant feature(target)

feature1 = gdf.columns[1]
feature2 = gdf.columns[8]
feature3 = gdf.columns[9]
feature4 = gdf.columns[3]
feature5 = gdf.columns[4]           # SepalLengthCm	SepalWidthCm	PetalLengthCm	PetalWidthCm	
target = gdf.columns[5]                      # Species 这里的features 对应选取多少个列，3，4，5，

print('Features:',feature1,feature2,feature3,feature4,feature5)
print('Target:',target)

# store feature matrix in "X"
X = gdf.iloc[:,[1,3,4,8,9]]                  # slicing: all rows and 1 to 2 cols （slicing:限制 这里选取对应的列，3个或4个，5个也行

# store response vector in "y"      
y = gdf.iloc[:,5]                            # slicing: all rows and 5th col 选取Mode 的列


print(X)
print(y.shape)

Features: altitude speed acceleration latitude longitude
Target: mode
         altitude   latitude   longitude      speed  acceleration
0          -777.0  39.894178  116.318200   0.000000  0.000000e+00
1          -777.0  39.894505  116.321132  26.827954  2.191826e-01
2          -777.0  39.894953  116.326452  27.927652  5.177485e-03
3          -777.0  39.894600  116.332542  31.870287  1.856231e-02
4          -777.0  39.889622  116.337040  41.090180  4.340816e-02
...           ...        ...         ...        ...           ...
5351928     289.0  40.029320  116.411975  41.772572 -2.795354e-02
5351929     275.0  40.029111  116.411963  41.812110  5.491364e-03
5351930     274.0  40.028904  116.411962  41.371997 -6.112673e-02
5351931     274.0  40.028697  116.411961  41.371996 -2.061781e-07
5351932     333.0  39.967705  116.412042   0.000000  0.000000e+00

[5351933 rows x 5 columns]
(5351933,)


In [27]:
# Convert response variable 'Species' to numbers and add this column  这里改个名字就行了，df改成gdf
# Necessary for the model training in the next step 

# Read more: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

# new col
gdf['EncodedSpecies'] = y

print('Classes:',le.classes_)
print('Response variable after encoding:',y)
gdf.head(10)

Classes: ['airplane' 'bike' 'boat' 'bus' 'car' 'motorcycle' 'run' 'subway' 'taxi'
 'train' 'walk']
Response variable after encoding: [9 9 9 ... 7 7 7]


,datetime,altitude,days_elapsed,latitude,longitude,mode,user_id,distance,speed,acceleration,geometry,EncodedSpecies
0,2008-03-28 14:54:40,-777.0,39535.621296,39.894178,116.318200,train,010,0.000000,0.000000,0.000000,POINT (116.31820 39.89418),9
1,2008-03-28 14:55:14,-777.0,39535.621690,39.894505,116.321132,train,010,253.375124,26.827954,0.219183,POINT (116.32113 39.89451),9
2,2008-03-28 14:56:13,-777.0,39535.622373,39.894953,116.326452,train,010,457.703187,27.927652,0.005177,POINT (116.32645 39.89495),9
3,2008-03-28 14:57:12,-777.0,39535.623056,39.894600,116.332542,train,010,522.318591,31.870287,0.018562,POINT (116.33254 39.89460),9
4,2008-03-28 14:58:11,-777.0,39535.623738,39.889622,116.337040,train,010,673.422399,41.090180,0.043408,POINT (116.33704 39.88962),9
5,2008-03-28 14:59:10,-777.0,39535.624421,39.882090,116.338353,train,010,843.803760,51.486331,0.048946,POINT (116.33835 39.88209),9
6,2008-03-28 15:00:10,-777.0,39535.625116,39.873873,116.338455,train,010,912.394041,54.743642,0.015080,POINT (116.33846 39.87387),9
7,2008-03-28 15:01:09,-777.0,39535.625799,39.865182,116.338058,train,010,965.577796,58.916611,0.019647,POINT (116.33806 39.86518),9
8,2008-03-28 15:02:08,-777.0,39535.626481,39.855403,116.337710,train,010,1086.189677,66.275980,0.034649,POINT (116.33771 39.85540),9
9,2008-03-28 15:03:07,-777.0,39535.627164,39.844532,116.334362,train,010,1240.567528,75.695646,0.044349,POINT (116.33436 39.84453),9


In [28]:
# Split X and y into training and testing sets

# Read more: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=4) #0.4 表示4—6分。取40% 
# 0.4这里改成0.1或0.2

In [29]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [30]:
# Search for the K observations in the training data that are "nearest" to  Class 是等于11 是mode
# the measurements of the unknown iris.

from sklearn.neighbors import KNeighborsClassifier

# Pick a value for K.
n_neighbors = 15

# create an instance of Neighbours Classifier and fit the data / train the model
knn = KNeighborsClassifier(n_neighbors)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='uniform')

In [31]:
#Evaluation classification accuracy

from sklearn import metrics

knn.fit(X_train, y_train)
k_pred = knn.predict(X_test)

metrics.accuracy_score(y_test, k_pred)

0.6827393431167016

In [3]:
# check best of K

from sklearn import metrics
v=[]
k_range = list(range(10, 20,)) #1和60我自己设置范围，这里可以多设一个step，因为数据太大了，在一个大范围里的高峰周围设置一个小范围，找到最高精度的K值


max_value = 0
max_k = 0
for i in k_range:
  knn = KNeighborsClassifier(n_neighbors=i)
  # fit the model with data
  knn.fit(X_train, y_train)
  k_pred = knn.predict(X_test)
  v.append(metrics.accuracy_score(y_test, k_pred))
  print('result of k = {}: {}'.format(i, v[-1]))

  if v[-1] > max_value:   #是for循环当前的结果与之前max的结果比较
    max_value = v[-1]
    max_k = i
  
  print('until now, max value is {} at k = {}'.format(max_value, max_k))
  print('-----------------------')

import matplotlib.pyplot as plt
plt.plot(k_range,v,c='Orange',)
plt.show()

NameError: name 'KNeighborsClassifier' is not defined